# Notes:
### This notebook loads, cleans, and restructures a CSV dataset containing information on Slovak household equivalent income intervals and their corresponding amounts and shares.

In [34]:
#import pandas

import pandas as pd

In [35]:
#Read in the data
data = pd.read_csv("./data/ps3808rr.csv",
    delimiter =",",
    skiprows=6, 
)

#Normalize column names (strip spaces)
data.columns = data.columns.astype(str).str.strip()

data.head()

,Code Element,Code Element.1,Code Element.2,Code Element.3,Value
0,SK0,2016,P_INT_spolu,mj_pocet_dom,1852058.0
1,SK0,2016,P_INT_spolu,mj_podiel_dom,100.0
2,SK0,2016,P_INT_1,mj_pocet_dom,29491.0
3,SK0,2016,P_INT_1,mj_podiel_dom,1.6
4,SK0,2016,P_INT_2,mj_pocet_dom,42331.0


In [36]:
#rename columns
data = data.rename(columns= {
    "Code Element": "region",
    "Code Element.1": "year",
    "Code Element.2": "intervals",
    "Code Element.3": "housholds",
    "Values": "Value",
    "mj_pocet_dom": "household_amount",
    "mj_podiel_dom": "household_share",
    
})

data.head()

,region,year,intervals,housholds,Value
0,SK0,2016,P_INT_spolu,mj_pocet_dom,1852058.0
1,SK0,2016,P_INT_spolu,mj_podiel_dom,100.0
2,SK0,2016,P_INT_1,mj_pocet_dom,29491.0
3,SK0,2016,P_INT_1,mj_podiel_dom,1.6
4,SK0,2016,P_INT_2,mj_pocet_dom,42331.0


In [37]:
# pivot the table
data = (
    data.pivot_table(
        index=["region", "year", "intervals"],
        columns="housholds",
        values="Value",
        aggfunc="first"
    )
    .reset_index()
)

# rename the new columns
data = data.rename(columns={
    "mj_pocet_dom": "household_amount",
    "mj_podiel_dom": "household_share"
})

# remove the *axis* name created by columns="housholds"
data = data.rename_axis(None, axis=1)  # drop columns axis name

# keep a plain numeric index
data = data.reset_index(drop=True)
data.index.name = None  # make sure index has no name

data.head()

,region,year,intervals,household_amount,household_share
0,SK0,2016,P_INT_1,29491.0,1.6
1,SK0,2016,P_INT_10,74214.0,4.0
2,SK0,2016,P_INT_11,138083.0,7.5
3,SK0,2016,P_INT_2,42331.0,2.3
4,SK0,2016,P_INT_3,68394.0,3.7


In [38]:
#rename values
region_mapping = {
    "SK0": "Slovak Republic",
    "SK01": "Region of Bratislava (NUTS2)",
    "SK010": "Region of Bratislava",
    "SK02": "West Slovakia",
    "SK021": "Region of Trnava",
    "SK022": "Region of Trenčín",
    "SK023": "Region of Nitra",
    "SK03": "Central Slovakia",
    "SK031": "Region of Žilina",
    "SK032": "Region of Banská Bystrica",
    "SK04": "East Slovakia",
    "SK041": "Region of Prešov",
    "SK042": "Region of Košice"
}

interval_mapping = {
    "P_INT_1": "<= 100",
    "P_INT_2": "101 - 200",
    "P_INT_3": "201 - 300",
    "P_INT_4": "301 - 400",
    "P_INT_5": "401 - 500",
    "P_INT_6": "501 - 600",
    "P_INT_7": "601 - 700",
    "P_INT_8": "701 - 800",
    "P_INT_9": "801 - 900",
    "P_INT_10": "901 - 1000",
    "P_INT_11": "1001 <",
    "P_INT_spolu": "total"
}

data["region"] = data["region"].replace(region_mapping)
data["intervals"] = data["intervals"].replace(interval_mapping)

data.head()

,region,year,intervals,household_amount,household_share
0,Slovak Republic,2016,<= 100,29491.0,1.6
1,Slovak Republic,2016,901 - 1000,74214.0,4.0
2,Slovak Republic,2016,1001 <,138083.0,7.5
3,Slovak Republic,2016,101 - 200,42331.0,2.3
4,Slovak Republic,2016,201 - 300,68394.0,3.7
